# Loading database

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///superstore.db

In [3]:
import pandas as pd
import sqlite3

In [4]:
conn = sqlite3.connect("superstore.db")
cursor = conn.cursor()
superstore_df = pd.read_csv("superstore_dataset.csv", encoding = "ISO-8859-1")

In [5]:
superstore_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [6]:
superstore_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [7]:
print(superstore_df['Customer ID'].nunique())
print(superstore_df['Product ID'].nunique())
print(superstore_df['Order ID'].nunique())

793
1862
5009


# Creating tables

## Creating Customers table

In [8]:
customers = superstore_df.groupby("Customer ID").agg({
    "Customer Name": "first",
    "Segment": "first",
    "Country": "first",
    "City": lambda x: x.value_counts().idxmax(),
    "State": lambda x: x.value_counts().idxmax(),
    "Postal Code": lambda x: x.value_counts().idxmax(),
    "Region": lambda x: x.value_counts().idxmax(),
}).reset_index()

In [9]:
customers.columns = [c.lower().replace(" ", "_") for c in customers.columns]

In [10]:
cursor.execute("DROP TABLE IF EXISTS customers;")
conn.commit()

In [11]:
cursor.execute("""
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_name TEXT,
    segment TEXT,
    country TEXT,
    city TEXT,
    state TEXT,
    postal_code INTEGER,
    region TEXT
);
""")
conn.commit()

In [12]:
customers.to_sql("customers", conn, if_exists = "append", index = False)

793

## Creating Products table

In [13]:
products = superstore_df.groupby("Product ID").agg({
    "Product Name": lambda x: x.value_counts().idxmax(),
    "Category": lambda x: x.value_counts().idxmax(),
    "Sub-Category": lambda x: x.value_counts().idxmax()
}).reset_index()

In [14]:
products.columns = [c.lower().replace(" ", "_").replace("-", "_") for c in products.columns]

In [15]:
cursor.execute("DROP TABLE IF EXISTS products;")
conn.commit()

In [16]:
cursor.execute("""
CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    sub_category TEXT
);
""")
conn.commit()

In [17]:
products.to_sql("products", conn, if_exists = "append", index = False)

1862

## Creating Orders table

In [18]:
orders = superstore_df.groupby("Order ID").agg({
    "Order Date": lambda x: x.value_counts().idxmax(),
    "Ship Date": lambda x: x.value_counts().idxmax(),
    "Ship Mode": lambda x: x.value_counts().idxmax(),
    "Customer ID": lambda x: x.value_counts().idxmax()
}).reset_index()

In [19]:
orders.columns = [c.lower().replace(" ", "_") for c in orders.columns]

In [20]:
orders["order_date"] = pd.to_datetime(orders["order_date"], format = "%m/%d/%Y").dt.strftime("%Y-%m-%d")
orders["ship_date"] = pd.to_datetime(orders["ship_date"], format = "%m/%d/%Y").dt.strftime("%Y-%m-%d")

In [21]:
cursor.execute("DROP TABLE IF EXISTS orders;")
conn.commit()

In [22]:
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute("""
CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    order_date TEXT,
    ship_date TEXT,
    ship_mode TEXT,
    customer_id TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
""")
conn.commit()

In [23]:
orders.to_sql("orders", conn, if_exists = "append", index = False)

5009

## Creating Order Details table

In [24]:
order_details = superstore_df[["Order ID", "Product ID", "Sales", "Quantity", "Discount", "Profit"]].copy()
order_details["Order Details ID"] = range(1, len(order_details) + 1)
len(order_details)

9994

In [25]:
order_details_duplicates = order_details[order_details.duplicated(keep = False)] 
print(len(order_details_duplicates), "\n", order_details_duplicates)

0 
 Empty DataFrame
Columns: [Order ID, Product ID, Sales, Quantity, Discount, Profit, Order Details ID]
Index: []


In [26]:
order_details.columns = [c.lower().replace(" ", "_") for c in order_details.columns]

In [27]:
cursor.execute("DROP TABLE IF EXISTS order_details;")
conn.commit()

In [28]:
cursor.execute("PRAGMA foreign_keys = ON;")
cursor.execute("""
CREATE TABLE order_details (
    order_details_id TEXT PRIMARY KEY,
    order_id TEXT,
    product_id TEXT,
    sales REAL,
    quantity INTEGER,
    discount REAL,
    profit REAL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
""")
conn.commit()

In [29]:
order_details.to_sql("order_details", conn, if_exists = "append", index = False)

9994

# Confirming each table is linked together properly

In [30]:
assert orders['customer_id'].isin(customers['customer_id']).all()
assert order_details['product_id'].isin(products['product_id']).all()
assert order_details['order_id'].isin(orders['order_id']).all()

# Verifying tables' data structure

In [31]:
query = """PRAGMA table_info(customers)"""
df = pd.read_sql_query(query, conn)
df

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,TEXT,0,None,1
1,1,customer_name,TEXT,0,None,0
2,2,segment,TEXT,0,None,0
3,3,country,TEXT,0,None,0
4,4,city,TEXT,0,None,0
5,5,state,TEXT,0,None,0
6,6,postal_code,INTEGER,0,None,0
7,7,region,TEXT,0,None,0


In [32]:
query = """PRAGMA table_info(products)"""
df = pd.read_sql_query(query, conn)
df

,cid,name,type,notnull,dflt_value,pk
0,0,product_id,TEXT,0,None,1
1,1,product_name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,sub_category,TEXT,0,None,0


In [33]:
query = """PRAGMA table_info(orders)"""
df = pd.read_sql_query(query, conn)
df

,cid,name,type,notnull,dflt_value,pk
0,0,order_id,TEXT,0,None,1
1,1,order_date,TEXT,0,None,0
2,2,ship_date,TEXT,0,None,0
3,3,ship_mode,TEXT,0,None,0
4,4,customer_id,TEXT,0,None,0


In [34]:
query = """PRAGMA table_info(order_details)"""
df = pd.read_sql_query(query, conn)
df

,cid,name,type,notnull,dflt_value,pk
0,0,order_details_id,TEXT,0,None,1
1,1,order_id,TEXT,0,None,0
2,2,product_id,TEXT,0,None,0
3,3,sales,REAL,0,None,0
4,4,quantity,INTEGER,0,None,0
5,5,discount,REAL,0,None,0
6,6,profit,REAL,0,None,0


In [35]:
conn.close()